# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# The following tables are loaded as pandas DataFrames with the same names: fct_guest_spending
# Filtering only July 2024 data 
july_data=fct_guest_spending[(fct_guest_spending["visit_date"].dt.year==2024) &
  (fct_guest_spending["visit_date"].dt.month==7)].copy()

#full list of park_experience_type
pet=pd.DataFrame(fct_guest_spending["park_experience_type"].unique(), columns=["park_experience_type"])

#aggregating by park_experience_type
avg_spending_byPET=july_data.groupby(by=["park_experience_type"],  as_index=False)["amount_spent"].mean()

#merging both and filling missing values 
result=pet.merge(avg_spending_byPET, on="park_experience_type", how="left").fillna(0.0).rename(columns={"amount_spent":"avg_spending_per_guest"})


print(result)

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
#filtering Aug 2024 data and guests with more than one visits
aug_data=fct_guest_spending[(fct_guest_spending["visit_date"].dt.year==2024) &
  (fct_guest_spending["visit_date"].dt.month==8)]
aug_data=aug_data[aug_data.groupby("guest_id")["guest_id"].transform('count')>1]

# print(aug_data.groupby("guest_id")["guest_id"].transform('count')>1)# returns booleans

#first visit
f_v=aug_data.groupby(by="guest_id", as_index=False)["visit_date"].min()
f_v_data=aug_data.merge(f_v, on=["guest_id","visit_date"], how="inner").rename(columns={"visit_date":"first_visit_date","amount_spent":"first_spending"})

#last visit 
l_v=aug_data.groupby(by="guest_id", as_index=False)["visit_date"].max()
l_v_data=aug_data.merge(l_v, on=["guest_id","visit_date"], how="inner").rename(columns={"visit_date":"last_visit_date","amount_spent":"last_spending"})

#merging both
final_data=f_v_data.merge(l_v_data, on="guest_id", how="left")

result_data=final_data[["guest_id", "first_spending", "last_spending"]]

result_data["diff_in_spending"]=result_data["first_spending"]-result_data["last_spending"]

result=result_data[["guest_id","diff_in_spending"]]
print(result)

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
#filtering data for sept 2024 and those who have not spent anything
sept_data=fct_guest_spending[(fct_guest_spending["visit_date"].dt.year==2024)&
  (fct_guest_spending["visit_date"].dt.month==9)]

sept_data=sept_data[sept_data["amount_spent"]!=0]

sept_data=sept_data.groupby(by="guest_id",as_index=False)["amount_spent"].sum()

#categorizing
conditions=[(sept_data["amount_spent"]>=0) & (sept_data["amount_spent"]<50),
             (sept_data["amount_spent"]>=50) & (sept_data["amount_spent"]<100)]

category=["Low", "Medium"]
                   
sept_data["category"]=np.select(conditions,category, default="High")
print(sept_data)

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)